> First time use: follow instructions in the README.md file in this directory.


'*[PT]** Português

---

**[EN]** English

# Instituto Nacional de Estatística / Instituto Hidrográfico
* https://snig.dgterritorio.gov.pt/rndg/srv/por/catalog.search#/metadata/3ec40a92-b050-4c47-bede-a2976debb8c7
*

## Toponímia de Portugal Continental
* Dados abertos
* Instituto Hidrográfico

* Data de Referência (Publicação): 07-12-2017
* Tema(s): Planeamento e Cadastro
* Cobertura: {"Portugal Continental":"Portugal Continental","Local":"Local"}
* Conjunto de dados Geográficos


* Acesso público sem restrições
* acesso e uso sem condições

* https://inspire.ine.pt/geoserver/gn/wfs?REQUEST=GetCapabilities&service=wfs&version=2.0.0
* https://inspire.ine.pt/geoserver/gn/wms?service=wms&version=1.3.0&request=GetCapabilities

Ver/See:
* https://docs.geoserver.org/stable/en/user/services/wfs/reference.html#wfs-getfeature
* https://www.quadratic.be/en/programmatically-interact-with-geoserver-in-python/ 




Install packages to access INE server and transform coordinates

In [1]:
!pip install requests


To convert coordinates to latitude longitude the proj library is needed.

First install proj library. See: https://proj.org/install.html

Then install pyproj in the next cell

In [2]:
!pip install requests


In [3]:

!pip install pyproj  # NOTE separateinstall of proj library needed see https://proj.org/install.html#install 

# Toponímia de Portugal Continental

Teste do servço WFS do INE

In [4]:
import requests
import json

n = 1
resp = requests.get(
                'https://inspire.ine.pt/geoserver/gn/wfs',
                params = {'request':'GetFeature',
                            'service':'wfs',
                            'version':'2.0.0',
                            'typeNames':'gn:NamedPlace',
                            'count':n,
                            'outputFormat':'json',
                            'exceptions':'application/json',
                            }
                )
print("URL:",resp.url)
resp_dict = json.loads(resp.content)
results = resp_dict.get('features',resp_dict.get('exceptions',[resp_dict]))
print(json.dumps(results[0], indent=2))

URL: https://inspire.ine.pt/geoserver/gn/wfs?request=GetFeature&service=wfs&version=2.0.0&typeNames=gn%3ANamedPlace&count=1&outputFormat=json&exceptions=application%2Fjson
{
  "type": "Feature",
  "id": "PT.GN.1",
  "geometry": {
    "type": "MultiPoint",
    "coordinates": [
      [
        -52673.206,
        190575.27
      ]
    ]
  },
  "properties": {
    "leastDetailedViewingResolution": {
      "MD_Resolution": {
        "equivalentScale": {
          "MD_RepresentativeFraction": {
            "denominator": {
              "Integer": 144447
            }
          }
        }
      }
    },
    "localType": {},
    "mostDetailedViewingResolution": {
      "MD_Resolution": {
        "equivalentScale": {
          "MD_RepresentativeFraction": {
            "denominator": {
              "Integer": 0
            }
          }
        }
      }
    },
    "name": {
      "GeographicalName": {
        "spelling": {
          "SpellingOfName": {
            "text": "P\u00d3VOA DE VA

Verificar se temos dados em cache.

In [280]:
from os.path import exists

import requests
import json
import pandas as pd
from pyproj import Transformer

cache_file = '../extras/geocoding/INE/ine_toponimia_clipped.csv'
if exists(cache_file):  # using data clipped to Portuguese border (in QGIS)
    cache_exists = True
    print("Locally cached clipped data found")
else:                  # no clipped version, use the original data 
    cache_file = '../extras/geocoding/INE/ine_toponimia.csv'
    if exists(cache_file):
        cache_exists = True
        print("Local cache data found")
    else:
        cache_exists = False
    
if not cache_exists:
    n=40000  #  as of june 2022 there are around 37k featues
    id='*'
    #  This could be done with geopandas by it is hard to install
    resp = requests.get(
                    'https://inspire.ine.pt/geoserver/gn/wfs',
                    params = {'request':'GetFeature',
                                # 'featureID':id,
                                'service':'wfs',
                                'version':'2.0.0',
                                'typeNames':'gn:NamedPlace',
                                'count':n,
                                # 'propertyName':'gn:*',
                                'outputFormat':'json',
                                'exceptions':'application/json',
                                }
                    )

    resp_dict = json.loads(resp.content)
    results = resp_dict.get('features',resp_dict.get('exceptions',[resp_dict]))
    # print(json.dumps(results[49], indent=2))
    tuples = []
    for f in results:
        id = f['id']
        name = f['properties']['name']['GeographicalName']['spelling']['SpellingOfName']['text']
        source = f['properties']['name']['GeographicalName']['sourceOfName']

        if f['geometry'].get('coordinates',None) is not None:
            coord = tuple(f['geometry']['coordinates'][0])
            x = coord[0]
            y = coord[1]
            # the coordinates from INE are projected, we convert to lat, long
            transformer = Transformer.from_crs('epsg:3763','epsg:4326')
            lat,long = transformer.transform(x,y)
        else:
            lat,long = None, None
        try:
            local_id = f['properties']['relatedSpatialObject']['Identifier']['localId']
        except(KeyError):
            local_id = None
        try:
            inspire_id = f['properties']['inspireId']['Identifier']['localId']
        except(KeyError):
            inspire_id = None
        try:
            local_type = f['properties']['localType']
            if len(local_type) > 0:
                local_type = str(local_type)
        except(KeyError):
            local_type = None
        try:
            inspire_type = f['properties']['type']
            if len(inspire_type) > 0:
                inspire_type = str(inspire_type)
        except(KeyError):
            inspire_type = None
        tuples = tuples + [(id, name, lat, long, source, local_type, inspire_type,local_id, inspire_id)]
        ine_df = pd.DataFrame(tuples, columns=['id','name','latitude','longitude','origin', 'local_type','inspire_type','local_id','inspire_id'])
        ine_df.to_csv(cache_file,index=False)
    print("Data loaded from INE server and saved to "+cache_file)
else:
    ine_df = pd.read_csv(cache_file, dtype={'inspire_id':'str'})
    print("Data loaded from local cache")

Locally cached clipped data found
Data loaded from local cache


Número de registos: 37.215 (Junho 2022).
Clipped: 36537


In [282]:
ine_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36357 entries, 0 to 36356
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          36357 non-null  object 
 1   name        36357 non-null  object 
 2   latitude    36357 non-null  float64
 3   longitude   36357 non-null  float64
 4   origin      36357 non-null  object 
 5   local_id    32507 non-null  object 
 6   inspire_id  29322 non-null  object 
dtypes: float64(2), object(5)
memory usage: 1.9+ MB


Exemplos

In [283]:
ine_df.sample(5)

,id,name,latitude,longitude,origin,local_id,inspire_id
16072,PT.GN.99049,Vascos,39.998796,-8.678775,INE,016458,BGRI2011_016458
763,PT.GN.1216,Aboim,41.326167,-8.069046,CIGEOE,NaN,5752
7534,PT.GN.90511,Sermil,41.625851,-8.493005,INE,004618,BGRI2011_004618
34046,PT.GN.117025,Bouças Novas,41.261213,-8.438940,DGT,CDG200k_populatedPlace4914,NaN
29078,PT.GN.112056,Formal,40.872601,-8.531016,INE,035725,BGRI2011_035725


Os registos provêm de diferentes origens e por isso contêm duplicações
(analisadas abaixo)

* CIGEOE: Centro de Informação Geográfica e Espacial do Exército (4539)
  * Usa abreviaturas: 
    * C.ça = Cabeça
* DGT: Direção Geral do Território, corresponde aos dados da carta 200k disponíveis também em separado (7054)
* IH: Instituto Hidrográfico, inclui "features" submarinas no Atlântico (149)
* INE: Toponímia do censo de 2011 (25.473)
  * O lugar consiste numa delimitação territorial, definida no âmbito das operações censitárias, que corresponde a um aglomerado populacional com dez ou mais alojamentos destinados à habitação de pessoas e com uma designação própria, independentemente de pertencer a uma ou mais freguesias. No âmbito dos Censos de 2011, foram recenseados cerca de 26 mil lugares no território nacional". https://www.ine.pt/xportal/xmain?xpid=INE&xpgid=ine_cont_inst&INST=6251013&xlang=pt
    


### Uniformização



In [668]:
abrev = {
  'V.Nova':'Vila Nova',
  'V.N.a': 'Nova',
  'V.NẂ': 'Nova',
  'V.Nª': 'Nova',
  'Rib.a':'Ribeira',
  'Sr.Ẃ':'Senhora',
  'Sr.a':'Senhora',
  'C.ça':'Cabeça',
  'S.ta':'Santa',
  'S.to':'Santo',
  'Sto.':'Santo',
  'M.te':'Monte',
  'F.te':'Fonte',
  'P.te':'Ponte',
  'Q.ta':'Quinta',
  'Srª':'Senhora',
  'N.Ẃ':'Nova',
  'V.e':'Vale',
  'C.':'Casal',
  'N.':'Nossa',
  'S.':'São',
  'V.':'Vila',
  " Dos ":" dos ",
  " Das ":" das ",
  " Do ":" do ",
  " Da ":" da ",
  " De ":" de "

}  


In [670]:
def unabrev(abrev,name):
    """Replace abreviations in string
    
    Args:
        abrev: dict with {abrev:expansion,...}
        name: string to return
    """

    nname = name
    if nname.isupper():
        nname = nname.title()
    for k,v in abrev.items():
        nname = nname.replace(k,v+' ')
    while "  " in nname:
        nname = nname.replace("  "," ")
    return nname

unabrev(abrev,"V.e de N. Sr.a da Anunciação em Rib.a do M.te da Q.ta do C. Novo junto ao F.te da P.te")

'Vila Nova de Famalicão'

In [671]:
ine_df['nname'] =ine_df['name'].apply(lambda name: unabrev(abrev, name))

In [678]:
ine_df[ine_df['name'] != ine_df['nname']].head(15)

,id,name,latitude,longitude,origin,local_id,inspire_id,nname
0,PT.GN.1,PÓVOA DE VARZIM,41.382737,-8.762818,CIGEOE,NaN,4537,Póvoa de Varzim
1,PT.GN.2,V. NOVA DE FAMALICÃO,41.407390,-8.520490,CIGEOE,NaN,4538,Vila Nova de Famalicão
2,PT.GN.3,GUIMARÃES,41.445545,-8.295324,CIGEOE,NaN,4539,Guimarães
3,PT.GN.4,FAFE,41.453953,-8.166989,CIGEOE,NaN,4540,Fafe
4,PT.GN.5,BARCELOS,41.534155,-8.617465,CIGEOE,NaN,4541,Barcelos
5,PT.GN.6,ESPOSENDE,41.532128,-8.776059,CIGEOE,NaN,4542,Esposende
6,PT.GN.7,CHAVES,41.742417,-7.470780,CIGEOE,NaN,4543,Chaves
7,PT.GN.8,VILA DO CONDE,41.354690,-8.743017,CIGEOE,NaN,4544,Vila do Conde
8,PT.GN.9,SANTO TIRSO,41.342612,-8.476255,CIGEOE,NaN,4545,Santo Tirso
9,PT.GN.10,FELGUEIRAS,41.367026,-8.198653,CIGEOE,NaN,4546,Felgueiras


In [679]:
ine_df.groupby('origin').count()

,id,name,latitude,longitude,local_id,inspire_id,nname
origin,,,,,,,
CIGEOE,3850,3850,3850,3850,0,3850,3850
DGT,7035,7035,7035,7035,7035,0,7035
INE,25472,25472,25472,25472,25472,25472,25472


## Ficheiro síntese do Censo de 2011

Inclui contagens de edifícios, famílias e pessoas com vários níveis de desagregação. Para os detalhes ver o ficheiro com a informação de síntese [extras/geocoding/INE/portugal2011/C2011_FSINTESE_VARIAVEIS.csv](../extras/geocoding/INE/portugal2011/C2011_FSINTESE_VARIAVEIS.csv).

Não parece conter toponímia abaixo da freguesia.

In [680]:
brigi11_pt_file = '../extras/geocoding/INE/portugal2011/BGRI11_PT.csv'
brigi11_df = pd.read_csv(brigi11_pt_file, sep=';', dtype={'GEO_COD':'str'}, encoding='iso-8859-1')
brigi11_df['GEO_COD']=brigi11_df['GEO_COD'].str.strip("'") # geo_doc contains a single non closed quote

In [287]:
keep =['ANO', 'GEO_COD', 'GEO_COD_DSG', 'NIVEL', 'NIVEL_DSG','N_INDIVIDUOS_RESIDENT','N_NUCLEOS_FAMILIARES']
drop = [c for c in brigi11_df.columns if c not in keep ] # don't need most cols
brigi11_df.drop(drop,axis=1, inplace=True)
brigi11_df.sample(5)


,ANO,GEO_COD,GEO_COD_DSG,NIVEL,NIVEL_DSG,N_NUCLEOS_FAMILIARES,N_INDIVIDUOS_RESIDENT
210100,2011,14010500215,,8,Subsecção,2,6
209107,2011,13180100428,,8,Subsecção,2,7
67325,2011,03124700302,,8,Subsecção,5,18
144737,2011,10130200216,,8,Subsecção,1,4
233666,2011,15080401803,,8,Subsecção,11,35


#### Sobre a organização espacial subjacente aos censos

"Geografia dos Censos


A preparação e a execução de um recenseamento geral da população e habitação exigem a referenciação geográfica da informação com base em unidades territoriais de pequena dimensão. Com esse intuito, o Instituto Nacional de Estatística desenvolveu um sistema de referenciação geográfica que, contendo a delimitação administrativa, divide as freguesias em pequenas áreas estatísticas – __secções__ e __subsecções__ estatísticas. Este sistema é integrado em formato digital na Base Geográfica de Referenciação de Informação (BGRI) que constitui uma infraestrutura fundamental de suporte às operações censitárias e, simultaneamente, um instrumento para a difusão de informação censitária.

Por __secção estatística__, entende-se a _unidade territorial correspondente a uma área contínua de uma única freguesia com cerca de 300 alojamentos destinados à habitação_. Por __subsecção estatística__, entende-se a _unidade territorial que identifica a mais pequena área homogénea de construção ou não, existente dentro da secção estatística_. Corresponde ao quarteirão nas áreas urbanas, ao lugar ou parte de um lugar nas áreas rurais, ou a áreas residuais que podem ou não conter alojamentos (isolados).

Em 2011, o território nacional estava dividido em cerca de 18 mil secções estatísticas e cerca de 266 mil subsecções estatísticas."
https://www.ine.pt/xportal/xmain?xpid=INE&xpgid=ine_cont_inst&INST=6251013&xlang=pt


In [288]:
brigi11_df.groupby(['NIVEL','NIVEL_DSG']).count()

,,ANO,GEO_COD,GEO_COD_DSG,N_NUCLEOS_FAMILIARES,N_INDIVIDUOS_RESIDENT
NIVEL,NIVEL_DSG,,,,,
1,Total Nacional,1,1,1,1,1
2,NUT1,3,3,3,3,3
3,NUT2,7,7,7,7,7
4,NUT3,30,30,30,30,30
5,Municipio,308,308,308,308,308
6,Freguesia,4260,4260,4260,4260,4260
7,Secção,18074,18074,18074,18074,18074
8,Subsecção,265955,265955,265955,265955,265955


Infelizmente os níveis inferiores não têm nome do topónimo
e não contêm um identificador comum com a informação de toponímia.

É contudo possível ligar com o ficheiro de toponímia através de 
informação nas Shapefiles do Censo 2011, como se verá adiante.

Até ao nível de freguesia os ficheiros do censo ainda contêm o nome do lugar (frequesia)

In [289]:
brigi11_df[brigi11_df.NIVEL == 6].sample(5)

,ANO,GEO_COD,GEO_COD_DSG,NIVEL,NIVEL_DSG,N_NUCLEOS_FAMILIARES,N_INDIVIDUOS_RESIDENT
3647,2011,160711,Cabaços,6,Freguesia,204,671
3057,2011,131120,Novelas,6,Freguesia,554,1794
732,2011,030252,Moure,6,Freguesia,282,925
2133,2011,090622,Vinhó,6,Freguesia,181,578
2001,2011,081404,Santa Catarina da Fonte do Bispo,6,Freguesia,584,1809


Para os níveis abaixo da frequesia não existe topónimo (GEO_COD_DSG)

In [290]:
brigi11_df[brigi11_df.NIVEL > 6].sample(5)

,ANO,GEO_COD,GEO_COD_DSG,NIVEL,NIVEL_DSG,N_NUCLEOS_FAMILIARES,N_INDIVIDUOS_RESIDENT
100451,2011,06161400510,,8,Subsecção,3,13
35098,2011,01131700311,,8,Subsecção,1,4
115588,2011,08100200104,,8,Subsecção,7,23
128160,2011,09110500109,,8,Subsecção,1,4
140016,2011,10091100221,,8,Subsecção,0,1


## Categorias administrativas do Censo de 2011

Códigos das freguesias, concelhos e distritos.
Existe um ficheiro disponível em http://mapas.ine.pt/download/files/2011/tabela/
http://mapas.ine.pt/download/files/2011/tabela/categorias.xlsx

Contudo esse ficheiro tem as categorias atuais e por
isso não corresponde aos códigos presentes nos dados do censo.

O ficheiro correspondente às categorias administrativas de 2011
consegue-se obter no site do INE em https://smi.ine.pt/Categoria/Exportacao?tipo=0
pesquisando pelo documento V00017 - Código da divisão administrativa (distritos/municípios/freguesias) 
e pedindo a versão de 2011.

In [15]:
!pip install openpyxl

In [291]:
cat2011_file = '../extras/geocoding/INE/categorias2011.xlsx'
cat2011_df = pd.read_excel(cat2011_file,skiprows=13, usecols=[1,2,3], dtype={'Código':'str'})
cat2011_df.head(10)


,Nível,Código,Designação
0,1,01,Aveiro
1,2,0101,Águeda
2,3,010101,Agadão
3,3,010102,Aguada de Baixo
4,3,010103,Aguada de Cima
5,3,010104,Águeda
6,3,010105,Barrô
7,3,010106,Belazaima do Chão
8,3,010107,Castanheira do Vouga
9,3,010108,Espinhel


Para cada categoria inserir os códigos e designações das categorias 
de nível superior em que estão inseridas e produzir uma descrição
estuturada "frequesia,concelho, distrito". Nessa descrição
estruturada simplifica-se a fórmula "União de freguesias..." 
(embora não existam nos dados de 2011, fica para o futuro)

In [483]:

cat2011_df['N1_CODE'] = None
cat2011_df['N2_CODE'] = None
cat2011_df['N3_CODE'] = None
cat2011_df['N1_DSG'] = None
cat2011_df['N2_DSG'] = None
cat2011_df['N3_DSG'] = None
cat2011_df['COD_DSG'] = None # this serves as a contextualized identifier 

for i,row in cat2011_df.sort_values('Código').iterrows():
    if row['Nível'] == 1:
        n1_code = row['Código']
        n1_dsg = row['Designação']
        cat2011_df.loc[i,'N1_CODE'] = n1_code
        cat2011_df.loc[i,'N1_DSG'] = n1_dsg
        cat2011_df.loc[i,'COD_DSG'] = n1_dsg
        cat2011_df.loc[i,'COD_UPPER_LEVEL'] = None

    elif  row['Nível'] == 2:
        n2_code = row['Código']
        n2_dsg = row['Designação']
        cat2011_df.loc[i,'N1_CODE'] = n1_code
        cat2011_df.loc[i,'N2_CODE'] = n2_code
        cat2011_df.loc[i,'N1_DSG'] = n1_dsg
        cat2011_df.loc[i,'N2_DSG'] = n2_dsg
        cat2011_df.loc[i,'COD_DSG'] = n2_dsg+ ', '+n1_dsg
        cat2011_df.loc[i,'COD_UPPER_LEVEL'] = n1_code
    elif  row['Nível'] == 3:
        n3_code = row['Código']
        n3_dsg = row['Designação'].removeprefix('União das freguesias de ').replace(", "," e ")
        cat2011_df.loc[i,'N1_CODE'] = n1_code
        cat2011_df.loc[i,'N2_CODE'] = n2_code
        cat2011_df.loc[i,'N3_CODE'] = n3_code
        cat2011_df.loc[i,'N1_DSG'] = n1_dsg
        cat2011_df.loc[i,'N2_DSG'] = n2_dsg
        cat2011_df.loc[i,'N3_DSG'] = n3_dsg
        cat2011_df.loc[i,'COD_DSG'] = n3_dsg + ', '+n2_dsg+ ', '+n1_dsg
        cat2011_df.loc[i,'COD_UPPER_LEVEL'] = n2_code

cat2011_df.sort_values('Código').head(15)

,Nível,Código,Designação,N1_CODE,N2_CODE,N3_CODE,N1_DSG,N2_DSG,N3_DSG,COD_DSG,COD_UPPER_LEVEL
0,1,01,Aveiro,01,None,None,Aveiro,None,None,Aveiro,NaN
1,2,0101,Águeda,01,0101,None,Aveiro,Águeda,None,"Águeda, Aveiro",01
2,3,010101,Agadão,01,0101,010101,Aveiro,Águeda,Agadão,"Agadão, Águeda, Aveiro",0101
3,3,010102,Aguada de Baixo,01,0101,010102,Aveiro,Águeda,Aguada de Baixo,"Aguada de Baixo, Águeda, Aveiro",0101
4,3,010103,Aguada de Cima,01,0101,010103,Aveiro,Águeda,Aguada de Cima,"Aguada de Cima, Águeda, Aveiro",0101
5,3,010104,Águeda,01,0101,010104,Aveiro,Águeda,Águeda,"Águeda, Águeda, Aveiro",0101
6,3,010105,Barrô,01,0101,010105,Aveiro,Águeda,Barrô,"Barrô, Águeda, Aveiro",0101
7,3,010106,Belazaima do Chão,01,0101,010106,Aveiro,Águeda,Belazaima do Chão,"Belazaima do Chão, Águeda, Aveiro",0101
8,3,010107,Castanheira do Vouga,01,0101,010107,Aveiro,Águeda,Castanheira do Vouga,"Castanheira do Vouga, Águeda, Aveiro",0101
9,3,010108,Espinhel,01,0101,010108,Aveiro,Águeda,Espinhel,"Espinhel, Águeda, Aveiro",0101


In [484]:
cat2011_file_out = '../extras/geocoding/INE/categorias2011.csv'
cat2011_df.to_csv(cat2011_file_out, index=None)

## Censos 2011 Principais dados alfanuméricos e geográficos

Contém dados e mapas do censo de 2011. A informação toponímica é a mesma da "Toponímia de Portugal Continental"
mas contém a estrutura administrativa e num ficheiro separado os
totais dos resultados do censo (edifícios, famílias e pessoas)

* http://mapas.ine.pt/download/files/

Este bloco de notas assume que o ficheiro em http://mapas.ine.pt/download/files/2011/portugal2011.zip 
foi transferido e descomprimido na directoria `extras/geocoding/INE/portugal2011`

In [18]:
!pip install dbf

In [293]:
from os.path import exists
import pandas as pd
import dbf

c2011_dbf = '../extras/geocoding/INE/portugal2011/CONTINENTE/BGRI11_CONT.dbf'
c2011_cache = '../extras/geocoding/INE/portugal2011/CONTINENTE/BGRI11_CONT.csv'

if exists(c2011_cache):
    cache_exists = True
    print("Local cache data found")
    c2011_df = pd.read_csv(c2011_cache, dtype= {
                'OBJECTID':'str',
                'DTMN11':'str',
                'FR11':'str',
                'SEC11':'str',
                'SS11':'str',
                'BGRI11':'str',
                'LUG11':'str',
                })
else:
    cache_exists = False
    with dbf.Table(c2011_dbf) as table:
        dbf.export(table,c2011_cache)
    c2011_df = pd.read_csv(c2011_cache, 
               dtype={
                'OBJECTID':'str',
                'DTMN11':'str',
                'FR11':'str',
                'SEC11':'str',
                'SS11':'str',
                'BGRI11':'str',
                'LUG11':'str',
                },
                encoding='iso-8859-1')
    c2011_df.to_csv(c2011_cache, index=False)
c2011_df.info()


Local cache data found
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 255844 entries, 0 to 255843
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   OBJECTID    255844 non-null  object
 1   DTMN11      255844 non-null  object
 2   FR11        255844 non-null  object
 3   SEC11       255844 non-null  object
 4   SS11        255844 non-null  object
 5   BGRI11      255844 non-null  object
 6   LUG11       255844 non-null  object
 7   LUG11DESIG  255844 non-null  object
dtypes: object(8)
memory usage: 15.6+ MB


In [294]:
c2011_df.sample(5)

,OBJECTID,DTMN11,FR11,SEC11,SS11,BGRI11,LUG11,LUG11DESIG
210586,211666,1809,03,003,07,18090300307,018508,Nelas ...
175878,170432,1101,16,001,22,11011600122,021308,Mato ...
231490,236911,1803,11,001,13,18031100113,018277,Eido ...
128275,122064,0808,08,018,11,08080801811,026230,Loulé ...
74421,72107,0312,46,001,04,03124600104,007042,Melhe ...


### Criação dos códigos administrativos compostos.

Os ficheiros do censo contém o código do concelho (__DTMN11__) e o número
da freguesia (__FR11__) dentro do concelho. 

Para obter o código completo da freguesia compatível com o ficheiro "categorias2011.xls"
é necessário concatenar os dois.

Aproveitamos e produzimos códigos qualificados das secções e dos lugares.

Não esquecer a definição de "Lugar" que corresponde ao _name place_ INSPIRE:

"Lugares

O lugar consiste numa delimitação territorial, definida no âmbito das operações censitárias, que corresponde a um aglomerado populacional com dez ou mais alojamentos destinados à habitação de pessoas e com uma designação própria, independentemente de pertencer a uma ou mais freguesias. No âmbito dos Censos de 2011, foram recenseados cerca de 26 mil lugares no território nacional"

https://www.ine.pt/xportal/xmain?xpid=INE&xpgid=ine_cont_inst&INST=6251013&xlang=pt


In [295]:
c2011_df['FR11ADMIN'] = c2011_df['DTMN11'].astype(str)+c2011_df['FR11'] 
c2011_df['SEC11ADMIN'] = c2011_df['DTMN11'].astype(str)+c2011_df['FR11']+c2011_df['SEC11']
c2011_df['LUG11ADMIN'] = c2011_df['LUG11'].astype(str)+'-'+c2011_df['FR11ADMIN']

In [296]:
c2011_df.head()

,OBJECTID,DTMN11,FR11,SEC11,SS11,BGRI11,LUG11,LUG11DESIG,FR11ADMIN,SEC11ADMIN,LUG11ADMIN
0,548,0407,02,001,04,04070200104,013810,Abreiro ...,040702,040702001,013810-040702
1,549,0407,02,001,06,04070200106,013810,Abreiro ...,040702,040702001,013810-040702
2,550,0407,02,001,01,04070200101,013811,Milhais ...,040702,040702001,013811-040702
3,551,0407,02,001,02,04070200102,013811,Milhais ...,040702,040702001,013811-040702
4,552,0407,29,001,03,04072900103,013866,Pai Torto ...,040729,040729001,013866-040729


O código composto __FR11ADMIN__ adicionado acima ao ficheiro do censo de 2011
é o mesmo que __N3_CODE__ do ficheiro de categorias, quando Nível == 3 (freguesia)

Atenção: é preciso usar o ficheiro de categorias em vigor em 2011 e não o que acompanha os 
mapas do Censo.

Retirando as freguesisas do Censo de 2011 e cruzando com as
categorias de 2011 (FR11ADMIN == N3_CODE) deve-se obter uma correspondência
perfeita.

In [297]:
c2011_fregs = c2011_df.sort_values('FR11ADMIN').drop_duplicates(subset='FR11ADMIN',keep='first')[['FR11ADMIN','DTMN11']]
c2011_fregs = c2011_fregs.merge(cat2011_df,how='left', left_on='FR11ADMIN', right_on='N3_CODE')
c2011_fregs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4050 entries, 0 to 4049
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   FR11ADMIN   4050 non-null   object
 1   DTMN11      4050 non-null   object
 2   Nível       4050 non-null   int64 
 3   Código      4050 non-null   object
 4   Designação  4050 non-null   object
 5   N1_CODE     4050 non-null   object
 6   N2_CODE     4050 non-null   object
 7   N3_CODE     4050 non-null   object
 8   N1_DSG      4050 non-null   object
 9   N2_DSG      4050 non-null   object
 10  N3_DSG      4050 non-null   object
 11  COD_DSG     4050 non-null   object
dtypes: int64(1), object(11)
memory usage: 411.3+ KB


In [298]:
c2011_fregs.sample(10)

,FR11ADMIN,DTMN11,Nível,Código,Designação,N1_CODE,N2_CODE,N3_CODE,N1_DSG,N2_DSG,N3_DSG,COD_DSG
3245,160504,1605,3,160504,Cossourado,16,1605,160504,Viana do Castelo,Paredes de Coura,Cossourado,"Cossourado, Paredes de Coura, Viana do Castelo"
3016,142002,1420,3,142002,Praia do Ribatejo,14,1420,142002,Santarém,Vila Nova da Barquinha,Praia do Ribatejo,"Praia do Ribatejo, Vila Nova da Barquinha, San..."
4003,182207,1822,3,182207,Vila Nova de Paiva,18,1822,182207,Viseu,Vila Nova de Paiva,Vila Nova de Paiva,"Vila Nova de Paiva, Vila Nova de Paiva, Viseu"
3935,181809,1818,3,181809,Freixinho,18,1818,181809,Viseu,Sernancelhe,Freixinho,"Freixinho, Sernancelhe, Viseu"
561,030725,0307,3,030725,Ribeiros,03,0307,030725,Braga,Fafe,Ribeiros,"Ribeiros, Fafe, Braga"
2759,131407,1314,3,131407,Carreira,13,1314,131407,Porto,Santo Tirso,Carreira,"Carreira, Santo Tirso, Porto"
1564,070908,0709,3,070908,Vera Cruz,07,0709,070908,Évora,Portel,Vera Cruz,"Vera Cruz, Portel, Évora"
2912,141102,1411,3,141102,Areias,14,1411,141102,Santarém,Ferreira do Zêzere,Areias,"Areias, Ferreira do Zêzere, Santarém"
1202,050420,0504,3,050420,Orca,05,0504,050420,Castelo Branco,Fundão,Orca,"Orca, Fundão, Castelo Branco"
636,030868,0308,3,030868,Silvares,03,0308,030868,Braga,Guimarães,Silvares,"Silvares, Guimarães, Braga"


### Merge de Toponímia do INE com a shape file do Censo 2011 para obter os códigos administrativos dos topónimos

Podemos cruzar a shapefile do Censo para a informação de toponímia do INE
usando os campos LUG11 e local_id ou alternativamente inspire_id.

ine.inspire_id == 'BRGRI2011_'+c2011.LUG11

ou 

ine.local_id = c2011.LUG11

Renomeamos as colinas para mais clareza.

In [681]:
ine_2011_df = ine_df.merge(c2011_df.sort_values('LUG11').drop_duplicates(subset='LUG11',keep='first'),how='left', left_on='local_id', right_on='LUG11')
ine_2011_df.drop(['OBJECTID','FR11','SEC11','SS11','LUG11','LUG11DESIG'], axis=1, inplace=True)
ine_2011_df.rename(columns={'DTMN11':'concelho_cod','FR11ADMIN':'freg_cod','SEC11ADMIN':'sec_cod','LUG11ADMIN':'lug_cod'}, inplace=True)


In [682]:
ine_2011_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36357 entries, 0 to 36356
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            36357 non-null  object 
 1   name          36357 non-null  object 
 2   latitude      36357 non-null  float64
 3   longitude     36357 non-null  float64
 4   origin        36357 non-null  object 
 5   local_id      32507 non-null  object 
 6   inspire_id    29322 non-null  object 
 7   nname         36357 non-null  object 
 8   concelho_cod  25472 non-null  object 
 9   BGRI11        25472 non-null  object 
 10  freg_cod      25472 non-null  object 
 11  sec_cod       25472 non-null  object 
 12  lug_cod       25472 non-null  object 
dtypes: float64(2), object(11)
memory usage: 3.9+ MB


Temos a correspondência entre lugar e freguesia (freg_cod) nos
topónimos cuja fonte é o INE.

In [683]:
ine_2011_df.sample(10)[['origin','name','local_id','freg_cod']]

,origin,name,local_id,freg_cod
2386,CIGEOE,São Cristóvão,NaN,NaN
7608,INE,Fundo de Vila,004731,031352
29155,INE,Monte Lemos,035810,080703
26009,INE,Sarnadas de Cima,030660,050601
3185,CIGEOE,Alqueve,NaN,NaN
34334,DGT,Cartomil,CDG200k_populatedPlace5223,NaN
34787,DGT,Chã,CDG200k_populatedPlace5176,NaN
19728,INE,Souropires,020478,091024
7175,INE,Bairro,004206,031017
10581,INE,Regadas,009534,130506


### Merge das categorias de 2011 com o ficheiro de toponímia

O campo freg_cod (FRE11) permite obter o contexto administrativo por extenso

In [687]:
ine_2011_cat_df = ine_2011_df.merge(cat2011_df.dropna(subset=['N3_CODE']),how='left',left_on='freg_cod',right_on='N3_CODE')
ine_2011_cat_df.drop(['BGRI11','Nível','Código','Designação','N2_CODE','N3_CODE'], axis=1, inplace=True)
ine_2011_cat_df.rename(columns={'N1_CODE':'distrito_cod','N1_DSG':'distrito_dsg','N2_DSG':'concelho_dsg','N3_DSG':'freg_dsg','COD_DSG':'admin_dsg'}, inplace=True)
ine_2011_cat_df['name_dsg'] = ine_2011_cat_df['nname'].astype(str)+', '+ine_2011_cat_df['admin_dsg']
ine_2011_cat_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36357 entries, 0 to 36356
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               36357 non-null  object 
 1   name             36357 non-null  object 
 2   latitude         36357 non-null  float64
 3   longitude        36357 non-null  float64
 4   origin           36357 non-null  object 
 5   local_id         32507 non-null  object 
 6   inspire_id       29322 non-null  object 
 7   nname            36357 non-null  object 
 8   concelho_cod     25472 non-null  object 
 9   freg_cod         25472 non-null  object 
 10  sec_cod          25472 non-null  object 
 11  lug_cod          25472 non-null  object 
 12  distrito_cod     25472 non-null  object 
 13  distrito_dsg     25472 non-null  object 
 14  concelho_dsg     25472 non-null  object 
 15  freg_dsg         25472 non-null  object 
 16  admin_dsg        25472 non-null  object 
 17  COD_UPPER_LE

#### Exemplo, concelho de Soure

Informação de Toponímia com o contexto administrativo.

In [688]:
ine_2011_cat_df[ine_2011_cat_df.concelho_cod == '0615'][['nname','name_dsg','latitude','longitude','inspire_id']]

,nname,name_dsg,latitude,longitude,inspire_id
15699,Alfarelos,"Alfarelos, Alfarelos, Soure, Coimbra",40.153030,-8.650008,BGRI2011_016050
15700,Casal do Redinho,"Casal do Redinho, Alfarelos, Soure, Coimbra",40.138151,-8.636902,BGRI2011_016051
15701,Brunhós,"Brunhós, Brunhós, Soure, Coimbra",40.113936,-8.669674,BGRI2011_016052
15702,Casais de S Jorge,"Casais de S Jorge, Degracias, Soure, Coimbra",40.005313,-8.532850,BGRI2011_016053
15703,Degracias,"Degracias, Degracias, Soure, Coimbra",40.009347,-8.520034,BGRI2011_016054
15704,Mocifas da Nazaré,"Mocifas da Nazaré, Degracias, Soure, Coimbra",39.997246,-8.526712,BGRI2011_016055
15705,Mocifas de Santo Amaro,"Mocifas de Santo Amaro, Degracias, Soure, Coimbra",39.990124,-8.529658,BGRI2011_016056
15706,Casal do Cimeiro,"Casal do Cimeiro, Figueiró do Campo, Soure, Co...",40.155912,-8.600338,BGRI2011_016057
15707,Casal de Marachão,"Casal de Marachão, Figueiró do Campo, Soure, C...",40.159115,-8.598061,BGRI2011_016058
15708,Casal das Neras,"Casal das Neras, Figueiró do Campo, Soure, Coi...",40.153593,-8.594173,BGRI2011_016059


### Merge do ficheiro de síntese do Censo de 2011 com o ficheiro de toponímia do INE

Podemos ligar os dados populacionais em brigi11_df à toponímia do INE
via o campo BGRI11 das shape files do Censo

ine.BGRI11 == c2011.BGRI11 == brigi11.GEO_COD

In [689]:
brigi11_df[brigi11_df.NIVEL > 7].head()

,ANO,GEO_COD,GEO_COD_DSG,NIVEL,NIVEL_DSG,N_EDIFICIOS_CLASSICOS,N_EDIFICIOS_CLASSICOS_1OU2,N_EDIFICIOS_CLASSICOS_ISOLADOS,N_EDIFICIOS_CLASSICOS_GEMIN,N_EDIFICIOS_CLASSICOS_EMBANDA,...,N_IND_RESID_DESEMP_PROC_EMPRG,N_IND_RESID_EMPREGADOS,N_IND_RESID_PENS_REFORM,N_IND_RESID_SEM_ACT_ECON,N_IND_RESID_EMPREG_SECT_PRIM,N_IND_RESID_EMPREG_SECT_SEQ,N_IND_RESID_EMPREG_SECT_TERC,N_IND_RESID_ESTUD_MUN_RESID,N_IND_RESID_TRAB_MUN_RESID,Unnamed: 127
22683,2011,01010100101,,8,Subsecção,8,8,8,0,0,...,0,4,3,3,0,3,1,2,3,NaN
22684,2011,01010100102,,8,Subsecção,8,8,8,0,0,...,0,9,5,10,0,6,3,0,5,NaN
22685,2011,01010100103,,8,Subsecção,9,9,9,0,0,...,1,6,7,9,0,2,4,2,6,NaN
22686,2011,01010100104,,8,Subsecção,3,3,3,0,0,...,0,0,4,4,0,0,0,0,0,NaN
22687,2011,01010100105,,8,Subsecção,5,5,5,0,0,...,0,2,5,10,1,0,1,4,2,NaN


Primeiro cruzamos com os mapas do censo de 2011 para obter o contexto administrativo.

In [690]:
bgrilug11_df = brigi11_df.merge(c2011_df.sort_values('BGRI11').drop_duplicates(subset='BGRI11',keep='first'),how='left', left_on='GEO_COD', right_on='BGRI11')
bgrilug11_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 288638 entries, 0 to 288637
Columns: 139 entries, ANO to LUG11ADMIN
dtypes: float64(1), int64(124), object(14)
memory usage: 308.3+ MB


In [307]:
bgrilug11_df[bgrilug11_df.NIVEL > 7].head()

,ANO,GEO_COD,GEO_COD_DSG,NIVEL,NIVEL_DSG,N_NUCLEOS_FAMILIARES,N_INDIVIDUOS_RESIDENT,OBJECTID,DTMN11,FR11,SEC11,SS11,BGRI11,LUG11,LUG11DESIG,FR11ADMIN,SEC11ADMIN,LUG11ADMIN
22683,2011,01010100101,,8,Subsecção,2,9,0,0101,01,001,01,01010100101,014420,Alcafaz ...,010101,010101001,014420-010101
22684,2011,01010100102,,8,Subsecção,7,19,0,0101,01,001,02,01010100102,014420,Alcafaz ...,010101,010101001,014420-010101
22685,2011,01010100103,,8,Subsecção,6,18,0,0101,01,001,03,01010100103,014421,Caselho ...,010101,010101001,014421-010101
22686,2011,01010100104,,8,Subsecção,2,4,0,0101,01,001,04,01010100104,014421,Caselho ...,010101,010101001,014421-010101
22687,2011,01010100105,,8,Subsecção,4,15,0,0101,01,001,05,01010100105,014421,Caselho ...,010101,010101001,014421-010101


Agregamos os dados ao nível do lugar. Só retemos os número de núcleos familiares e número de indivíduos.


In [691]:
lug11_pop = bgrilug11_df.groupby('LUG11').agg(nucleos_familiares = ('N_NUCLEOS_FAMILIARES','sum'),residentes = ('N_INDIVIDUOS_RESIDENT','sum')).reset_index()
lug11_pop.nucleos_familiares = lug11_pop.nucleos_familiares.astype(int)
lug11_pop.residentes = lug11_pop.residentes.astype(int)
lug11_pop.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25474 entries, 0 to 25473
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   LUG11               25474 non-null  object
 1   nucleos_familiares  25474 non-null  int64 
 2   residentes          25474 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 597.2+ KB


In [692]:

lug11_pop.sample(10)

,LUG11,nucleos_familiares,residentes
17845,022779,203,629
20885,026374,76,233
2787,003396,28,98
4202,005405,44,143
17363,022110,10,29
1375,001640,21,70
24072,034186,74,221
19220,024367,260,833
11705,015892,22,61
4485,005821,10,25


Copiamos os dados demográfico para a
lista de topónimos.

O resultado é uma lista de topónimos
com contexto administrativo e 
resultados censo de 2011.

In [693]:
ine_2011_pop = ine_2011_cat_df.merge(lug11_pop, how='left', left_on='local_id', right_on='LUG11')
ine_2011_pop.drop(['LUG11'], axis=1, inplace=True)
ine_2011_pop.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36357 entries, 0 to 36356
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  36357 non-null  object 
 1   name                36357 non-null  object 
 2   latitude            36357 non-null  float64
 3   longitude           36357 non-null  float64
 4   origin              36357 non-null  object 
 5   local_id            32507 non-null  object 
 6   inspire_id          29322 non-null  object 
 7   nname               36357 non-null  object 
 8   concelho_cod        25472 non-null  object 
 9   freg_cod            25472 non-null  object 
 10  sec_cod             25472 non-null  object 
 11  lug_cod             25472 non-null  object 
 12  distrito_cod        25472 non-null  object 
 13  distrito_dsg        25472 non-null  object 
 14  concelho_dsg        25472 non-null  object 
 15  freg_dsg            25472 non-null  object 
 16  admi

## Exemplos dos dados consolidados

Toponímia, coordenadas, contexto administrativo, demografia dos censos.

##### Lisboa

In [694]:
ine_2011_pop[ine_2011_pop.inspire_id =='BGRI2011_022406']

,id,name,latitude,longitude,origin,local_id,inspire_id,nname,concelho_cod,freg_cod,...,lug_cod,distrito_cod,distrito_dsg,concelho_dsg,freg_dsg,admin_dsg,COD_UPPER_LEVEL,name_dsg,nucleos_familiares,residentes
21432,PT.GN.104409,Lisboa,38.741645,-9.157559,INE,022406,BGRI2011_022406,Lisboa,1106,110643,...,022406-110643,11,Lisboa,Lisboa,São João de Deus,"São João de Deus, Lisboa, Lisboa",1106,"Lisboa, São João de Deus, Lisboa, Lisboa",154519.0,547733.0


##### Soure

In [695]:
import pandas as pd
pd.set_option('display.max_rows',200)

ine_2011_pop[ine_2011_pop.concelho_cod == '0615'][['id','name','longitude','latitude','nucleos_familiares','residentes','name_dsg','inspire_id']].sort_values('residentes', ascending=False)

,id,name,longitude,latitude,nucleos_familiares,residentes,name_dsg,inspire_id
15790,PT.GN.98767,Soure,-8.628098,40.059784,544.0,1831.0,"Soure, Soure, Soure, Coimbra",BGRI2011_016146
15719,PT.GN.98696,Granja do Ulmeiro,-8.633364,40.160125,513.0,1606.0,"Granja do Ulmeiro, Granja do Ulmeiro, Soure, C...",BGRI2011_016071
15699,PT.GN.98676,Alfarelos,-8.650008,40.153030,320.0,1036.0,"Alfarelos, Alfarelos, Soure, Coimbra",BGRI2011_016050
15745,PT.GN.98722,Vila Nova de Anços,-8.633647,40.110361,314.0,985.0,"Vila Nova de Anços, Vila Nova de Anços, Soure,...",BGRI2011_016099
15710,PT.GN.98687,Figueiró do Campo,-8.579403,40.147902,191.0,616.0,"Figueiró do Campo, Figueiró do Campo, Soure, C...",BGRI2011_016061
15789,PT.GN.98766,Sobral,-8.649280,40.031900,174.0,548.0,"Sobral, Soure, Soure, Coimbra",BGRI2011_016145
15784,PT.GN.98761,Pouca Pena,-8.604650,40.100687,138.0,414.0,"Pouca Pena, Soure, Soure, Coimbra",BGRI2011_016140
15782,PT.GN.98759,Paleão,-8.597359,40.048394,135.0,403.0,"Paleão, Soure, Soure, Coimbra",BGRI2011_016137
15788,PT.GN.98765,Simões,-8.633455,40.007501,108.0,344.0,"Simões, Soure, Soure, Coimbra",BGRI2011_016144
15713,PT.GN.98690,Cercal,-8.668092,40.098317,99.0,341.0,"Cercal, Gesteira, Soure, Coimbra",BGRI2011_016065


##### Prazo dos Estudantes

Registo consolidado

In [696]:
name = 'Prazo dos Estudantes'

ine_2011_pop[ine_df.name == name].iloc[0]

id                                                   PT.GN.111697
name                                         Prazo dos Estudantes
latitude                                                40.053108
longitude                                               -8.614399
origin                                                        INE
local_id                                                   035183
inspire_id                                        BGRI2011_035183
nname                                        Prazo dos Estudantes
concelho_cod                                                 0615
freg_cod                                                   061509
sec_cod                                                 061509005
lug_cod                                             035183-061509
distrito_cod                                                   06
distrito_dsg                                              Coimbra
concelho_dsg                                                Soure
freg_dsg  

Informação no Censo de 2011, subsecção

In [697]:
c2011_df[c2011_df.LUG11.isin(ine_df[ine_df.name == name]['local_id'])].sort_values('BGRI11')


,OBJECTID,DTMN11,FR11,SEC11,SS11,BGRI11,LUG11,LUG11DESIG,FR11ADMIN,SEC11ADMIN,LUG11ADMIN
18814,29717,0615,09,005,05,06150900505,035183,Prazo dos Estudantes ...,061509,061509005,035183-061509
17727,29408,0615,09,005,06,06150900506,035183,Prazo dos Estudantes ...,061509,061509005,035183-061509


Dados do Censo, do ficheiro de síntese

In [698]:
geo_codes = c2011_df[c2011_df.LUG11.isin(ine_df[ine_df.name == name]['local_id'])]['BGRI11']
print(geo_codes)
bgrilug11_df[bgrilug11_df.GEO_COD.isin(geo_codes)]

17727    06150900506
18814    06150900505
Name: BGRI11, dtype: object


,ANO,GEO_COD,GEO_COD_DSG,NIVEL,NIVEL_DSG,N_EDIFICIOS_CLASSICOS,N_EDIFICIOS_CLASSICOS_1OU2,N_EDIFICIOS_CLASSICOS_ISOLADOS,N_EDIFICIOS_CLASSICOS_GEMIN,N_EDIFICIOS_CLASSICOS_EMBANDA,...,DTMN11,FR11,SEC11,SS11,BGRI11,LUG11,LUG11DESIG,FR11ADMIN,SEC11ADMIN,LUG11ADMIN
99320,2011,06150900505,,8,Subsecção,9,9,8,1,0,...,0615,09,005,05,06150900505,035183,Prazo dos Estudantes ...,061509,061509005,035183-061509
99321,2011,06150900506,,8,Subsecção,9,9,7,2,0,...,0615,09,005,06,06150900506,035183,Prazo dos Estudantes ...,061509,061509005,035183-061509


Ficheiro de toponímia consolidado, acesso por nome geográfico

In [699]:
ine_2011_pop[ine_2011_pop.name == name]

,id,name,latitude,longitude,origin,local_id,inspire_id,nname,concelho_cod,freg_cod,...,lug_cod,distrito_cod,distrito_dsg,concelho_dsg,freg_dsg,admin_dsg,COD_UPPER_LEVEL,name_dsg,nucleos_familiares,residentes
28719,PT.GN.111697,Prazo dos Estudantes,40.053108,-8.614399,INE,035183,BGRI2011_035183,Prazo dos Estudantes,0615,061509,...,035183-061509,06,Coimbra,Soure,Soure,"Soure, Soure, Coimbra",0615,"Prazo dos Estudantes, Soure, Soure, Coimbra",15.0,51.0


Dados da secção estatística que inclui o topónimo.

In [700]:
brigi11_df[brigi11_df.GEO_COD == '061509005']

,ANO,GEO_COD,GEO_COD_DSG,NIVEL,NIVEL_DSG,N_EDIFICIOS_CLASSICOS,N_EDIFICIOS_CLASSICOS_1OU2,N_EDIFICIOS_CLASSICOS_ISOLADOS,N_EDIFICIOS_CLASSICOS_GEMIN,N_EDIFICIOS_CLASSICOS_EMBANDA,...,N_IND_RESID_DESEMP_PROC_EMPRG,N_IND_RESID_EMPREGADOS,N_IND_RESID_PENS_REFORM,N_IND_RESID_SEM_ACT_ECON,N_IND_RESID_EMPREG_SECT_PRIM,N_IND_RESID_EMPREG_SECT_SEQ,N_IND_RESID_EMPREG_SECT_TERC,N_IND_RESID_ESTUD_MUN_RESID,N_IND_RESID_TRAB_MUN_RESID,Unnamed: 127
8821,2011,061509005,,7,Secção,411,406,316,69,21,...,33,285,218,343,6,56,223,92,157,NaN


In [701]:
bgrilug11_df[bgrilug11_df.SEC11ADMIN == '061509005']

,ANO,GEO_COD,GEO_COD_DSG,NIVEL,NIVEL_DSG,N_EDIFICIOS_CLASSICOS,N_EDIFICIOS_CLASSICOS_1OU2,N_EDIFICIOS_CLASSICOS_ISOLADOS,N_EDIFICIOS_CLASSICOS_GEMIN,N_EDIFICIOS_CLASSICOS_EMBANDA,...,DTMN11,FR11,SEC11,SS11,BGRI11,LUG11,LUG11DESIG,FR11ADMIN,SEC11ADMIN,LUG11ADMIN
99316,2011,06150900501,,8,Subsecção,11,10,10,0,0,...,0615,09,005,01,06150900501,016156,Novos ...,061509,061509005,016156-061509
99317,2011,06150900502,,8,Subsecção,14,14,11,2,1,...,0615,09,005,02,06150900502,016156,Novos ...,061509,061509005,016156-061509
99318,2011,06150900503,,8,Subsecção,6,5,3,2,0,...,0615,09,005,03,06150900503,016146,Soure ...,061509,061509005,016146-061509
99319,2011,06150900504,,8,Subsecção,2,2,2,0,0,...,0615,09,005,04,06150900504,016146,Soure ...,061509,061509005,016146-061509
99320,2011,06150900505,,8,Subsecção,9,9,8,1,0,...,0615,09,005,05,06150900505,035183,Prazo dos Estudantes ...,061509,061509005,035183-061509
99321,2011,06150900506,,8,Subsecção,9,9,7,2,0,...,0615,09,005,06,06150900506,035183,Prazo dos Estudantes ...,061509,061509005,035183-061509
99322,2011,06150900507,,8,Subsecção,3,3,3,0,0,...,0615,09,005,07,06150900507,016143,Quinta de S. Mateus ...,061509,061509005,016143-061509
99323,2011,06150900508,,8,Subsecção,4,4,4,0,0,...,0615,09,005,08,06150900508,016143,Quinta de S. Mateus ...,061509,061509005,016143-061509
99324,2011,06150900509,,8,Subsecção,6,6,6,0,0,...,0615,09,005,09,06150900509,016143,Quinta de S. Mateus ...,061509,061509005,016143-061509
99325,2011,06150900510,,8,Subsecção,2,2,2,0,0,...,0615,09,005,10,06150900510,016143,Quinta de S. Mateus ...,061509,061509005,016143-061509


## Exportar os dados consolidados


In [702]:
ine_2011_pop.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36357 entries, 0 to 36356
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  36357 non-null  object 
 1   name                36357 non-null  object 
 2   latitude            36357 non-null  float64
 3   longitude           36357 non-null  float64
 4   origin              36357 non-null  object 
 5   local_id            32507 non-null  object 
 6   inspire_id          29322 non-null  object 
 7   nname               36357 non-null  object 
 8   concelho_cod        25472 non-null  object 
 9   freg_cod            25472 non-null  object 
 10  sec_cod             25472 non-null  object 
 11  lug_cod             25472 non-null  object 
 12  distrito_cod        25472 non-null  object 
 13  distrito_dsg        25472 non-null  object 
 14  concelho_dsg        25472 non-null  object 
 15  freg_dsg            25472 non-null  object 
 16  admi

In [703]:

ine_topo_file = cache_file
ine_2011_pop.to_csv(ine_topo_file, index=False)
print("Exporting to", cache_file)

Exporting to ../extras/geocoding/INE/ine_toponimia_clipped.csv


## Análise do ficheiro consolidado

In [47]:
! pip install folium

### Análise dos identificadores

In [704]:
cigoe_df = ine_2011_pop[ine_2011_pop.origin=='CIGEOE']
dgt_df = ine_2011_pop[ine_2011_pop.origin=='DGT']
ine_ine_df = ine_2011_pop[ine_2011_pop.origin=='INE']

cigoe_names = cigoe_df['name'].str.lower()
dgt_names = dgt_df['name'].str.lower()
ine_ine_names = ine_ine_df['name'].str.lower()

print("Number of names in cigoe:",len(cigoe_names))
print("Number of names in dgt:",len(dgt_names))
print("Number of names in ine:",len(ine_ine_names))

Number of names in cigoe: 3850
Number of names in dgt: 7035
Number of names in ine: 25472


### Topo-homonomia

In [705]:
ine_2011_pop.groupby('name').agg(count=('name','count')).reset_index().sort_values('count', ascending=False).head(15)

,name,count
6824,Igreja,349
9074,Outeiro,291
8335,Monte,160
10227,Portela,135
12756,Souto,126
3003,Carvalhal,115
1947,Boavista,105
371,Aldeia,104
9487,Paço,91
3274,Casal,90


### Sobreposição das fontes de dados

In [706]:
common_dgt_cigoe = list(set(cigoe_names).intersection(set(dgt_names)))
common_ine_cigoe = list(set(cigoe_names).intersection(set(ine_ine_names)))
common_ine_cigoe_dgt = list(set(cigoe_names).intersection(set(ine_ine_names)).intersection(set(dgt_names)))
print("Common names cigoe-dgt", len(common_dgt_cigoe))
print("Common names cigoe-ine", len(common_ine_cigoe))
print("Common names cigoe-ine-dgt", len(common_ine_cigoe_dgt))
print(common_ine_cigoe_dgt[:20])



Common names cigoe-dgt 1805
Common names cigoe-ine 2591
Common names cigoe-ine-dgt 1796
['samil', 'palmela', 'cochadas', 'figueira', 'moimenta', 'arcozelo', 'póvoa de atalaia', 'marmeleiro', 'apúlia', 'carrazeda de ansiães', 'fuseta', 'mamouros', 'celorico de basto', 'runa', 'aguada de baixo', 'almagreira', 'covão', 'mouriscas', 'fornelos', 'antas']


## Remoção de topónimos redundantes

Removemos os topónimos que:
* não tenham origem no INE
* estejam localizados menos de um km de um topónimo do INE com o mesmo nome.
* quando vários topónimos do INE estão a menos de 1KM, escolhe-se o mais próximo



In [50]:
! pip install recordlinkage

We need geopy to calculate distances

In [220]:
! pip install geopy

  Using cached geopy-2.2.0-py3-none-any.whl (118 kB)
  Using cached geographiclib-1.52-py3-none-any.whl (38 kB)


### Assinalar topónimos com o mesmo nome a menos de 1km de distância

Alguns misses:

* DGT: Braga, distância a INE: cerca de 3km

In [709]:
import recordlinkage
from geopy.distance import geodesic

threshold = 1.5

origin, dataframe = ('CIGEOE',cigoe_df)

for origin, dataframe in [('CIGEOE',cigoe_df), ('DGT',dgt_df)]:
    print("Processing:",origin)
    indexer = recordlinkage.index.Block(left_on='nname', right_on='nname')
    candidates = indexer.index(ine_ine_df,dataframe)
    print(len(candidates))
    compare = recordlinkage.Compare()
    compare.string('nname','nname',method='jarowinkler', label='same_name')
    compare.geo( label='same_location',   # this seems to match up to 1km
                left_on_lat='latitude',   # not sure how to increase
                left_on_lng='longitude',
                right_on_lat='latitude',
                right_on_lng='longitude',
                method='linear'
                )
    candidates = compare.compute(candidates,ine_ine_df,dataframe)
    candidates.index.names = ('INE',origin)
    candidates['total'] = candidates.sum(axis =1)
    # we filter
    candidates = candidates[candidates.total > threshold].reset_index()
    candidates.sort_values('total', ascending=False, inplace=True)
    candidates.drop_duplicates(subset=origin, keep='first', inplace=True)
    candidates.set_index(['INE',origin], inplace=True)
    #
    # candidates[candidates.same_location>0].hist(column='total', bins=150)
    # candidates[candidates.same_location>0].describe()
    #
    # as we go by increasing order of proximity, in case of multiple matches
    #   the closest ones win.
    for (l,r), row in candidates.sort_values('same_location').iterrows():
        left = ine_2011_pop.iloc[l]
        right = ine_2011_pop.iloc[r]
        dist_meters = geodesic((left.latitude,left.longitude), (right.latitude,right.longitude)).km*1000
        ine_2011_pop.loc[r,'duplicate'] = True
        ine_2011_pop.loc[r,'same_as'] = ine_2011_pop.iloc[l]['id']
        ine_2011_pop.loc[r,'same_as_score'] = row['same_location']
        ine_2011_pop.loc[r,'same_as_distance'] = dist_meters


Processing: CIGEOE
20456
Processing: DGT
83261


In [710]:

ine_2011_pop.info()  


<class 'pandas.core.frame.DataFrame'>
Int64Index: 36357 entries, 0 to 36356
Data columns (total 25 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  36357 non-null  object 
 1   name                36357 non-null  object 
 2   latitude            36357 non-null  float64
 3   longitude           36357 non-null  float64
 4   origin              36357 non-null  object 
 5   local_id            32507 non-null  object 
 6   inspire_id          29322 non-null  object 
 7   nname               36357 non-null  object 
 8   concelho_cod        25472 non-null  object 
 9   freg_cod            25472 non-null  object 
 10  sec_cod             25472 non-null  object 
 11  lug_cod             25472 non-null  object 
 12  distrito_cod        25472 non-null  object 
 13  distrito_dsg        25472 non-null  object 
 14  concelho_dsg        25472 non-null  object 
 15  freg_dsg            25472 non-null  object 
 16  admi

### Análise dos topónimos autónomos do CIGEOE e DGT

Em Junho 2022 o CIGEOE acrescenta 1048 topónimos 
aos dados do INE e a DGT 334.

In [711]:
ine_2011_pop.fillna({'duplicate':False}).groupby(['origin','duplicate'])['id'].count()

origin  duplicate
CIGEOE  False         1046
        True          2804
DGT     False          334
        True          6701
INE     False        25472
Name: id, dtype: int64

Os topónimos da DGT incluem também 
formações naturais (cabos, ilhas, pontas, serras)
e construções como santuários e fortes.

É possível inferir o tipo do __local_id__ incluíndo
nos dados.

Conclui-se que apenas 73 topónimos correspondem a __populatedPlace__.

In [407]:
dgt_filter = ine_2011_pop.origin == 'DGT'
ine_2011_pop.loc[dgt_filter,'dgt_type'] = ine_2011_pop.loc[dgt_filter,'local_id'].str.strip("CDG_k0123456789")

In [411]:
ine_2011_pop[(ine_2011_pop.origin == 'DGT') & (ine_2011_pop.duplicate.isnull())].groupby('dgt_type')['dgt_type'].count()

dgt_type
buildingForte         23
buildingSantuario     13
landformCabo           7
landformIlha          15
landformPonta         54
landformSerra        153
populatedPlace        73
Name: dgt_type, dtype: int64

Os lugares que não são __populatedPlace__ não devem
participar da desduplicação.

In [584]:
dgt_nonpop = (ine_2011_pop.origin == 'DGT') & (ine_2011_pop.dgt_type != 'populatedPlace')
ndx = ine_2011_pop[dgt_nonpop].index.values
ine_2011_pop.loc[ndx,'duplicate'] = False
ine_2011_pop.loc[ndx][['name','origin','dgt_type','duplicate']]

,name,origin,dgt_type,duplicate
35821,Alcáçovas,DGT,landformSerra,False
35822,Alcaria Ruiva,DGT,landformSerra,False
35823,Alfeizeirao,DGT,landformSerra,False
35824,Alqueidao,DGT,landformSerra,False
35825,ALVAO,DGT,landformSerra,False
...,...,...,...,...
36352,Candeeiras,DGT,landformPonta,False
36353,Alheta,DGT,landformPonta,False
36354,Sagres,DGT,landformPonta,False
36355,Atalaia,DGT,landformPonta,False


Amostra de topónimos de locais povoados
nos dados da DGT que não estão nos dados do INE.

In [603]:
ine_2011_pop[(ine_2011_pop.dgt_type == 'populatedPlace') & (ine_2011_pop.duplicate.isnull())].sample(5)

,id,name,latitude,longitude,origin,local_id,inspire_id,concelho_cod,freg_cod,sec_cod,...,duplicate,same_as,same_as_score,same_as_distance,pop_rank_in_name,dist_bonus,concelho_bonus,freg_bonus,importance,dgt_type
36193,PT.GN.119187,Vinhais,41.822420,-7.003587,DGT,CDG200k_populatedPlace6660,NaN,0412,041235,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,1.0,1.0,NaN,populatedPlace
29675,PT.GN.112654,Quinta do Anjo,38.561515,-8.940780,DGT,CDG200k_populatedPlace625,NaN,1508,150804,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,NaN,populatedPlace
32327,PT.GN.115306,Oliveira do Conde,40.425944,-7.966724,DGT,CDG200k_populatedPlace3132,NaN,1802,180204,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,NaN,populatedPlace
31194,PT.GN.114173,Alcanadas,39.624656,-8.797689,DGT,CDG200k_populatedPlace1835,NaN,1016,101602,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,populatedPlace
30686,PT.GN.113665,Atalaia,39.218905,-8.773512,DGT,CDG200k_populatedPlace1407,NaN,1416,141616,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,populatedPlace


Amostra de topónimos unicamente nos dados do CIGEOE

In [587]:
ine_2011_pop[(ine_2011_pop.origin == 'CIGEOE') & (ine_2011_pop.duplicate.isnull())].sample(30)

,id,name,latitude,longitude,origin,local_id,inspire_id,concelho_cod,freg_cod,sec_cod,...,duplicate,same_as,same_as_score,same_as_distance,pop_rank_in_name,dist_bonus,concelho_bonus,freg_bonus,importance,dgt_type
2202,PT.GN.2655,Parceiros da Igreja,39.453710,-8.609547,CIGEOE,NaN,7191,1419,141908,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN
2672,PT.GN.3126,S. Pedro de Sólis,37.493937,-7.900285,CIGEOE,NaN,7662,0209,020908,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN
53,PT.GN.54,ALMADA,38.683895,-9.152606,CIGEOE,NaN,4590,1503,150306,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN
2917,PT.GN.3372,Aldeia Nova,41.553572,-6.232249,CIGEOE,NaN,7908,0406,040608,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN
359,PT.GN.812,S.ta Marta da Montanha,41.503241,-7.741672,CIGEOE,NaN,5348,1713,171309,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN
1996,PT.GN.2449,Pêro Martins,40.881549,-7.085159,CIGEOE,NaN,6985,0904,090412,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN
103,PT.GN.134,ALVERCA DO RIBATEJO,38.898391,-9.033562,CIGEOE,NaN,4670,1114,111402,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN
3055,PT.GN.3510,Aguda,41.050319,-8.653840,CIGEOE,NaN,8046,1317,131701,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN
1870,PT.GN.2323,Maçal,40.692956,-7.301653,CIGEOE,NaN,6859,0903,090309,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN
898,PT.GN.1351,Ovil,41.180654,-8.016030,CIGEOE,NaN,5887,1302,130211,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,NaN,NaN


### Determinação contexto administrativo aos topónimos do CIGEOE e DGT

Essa atribuição pode ser relevante para utilizar o cálculo da importância
e na criação de designações não ambíguas.

Para isso localizamos a freguesia dentro da qual os topónimos se
localizam.


In [418]:
! pip install shapely

  Using cached Shapely-1.8.2-cp39-cp39-macosx_11_0_arm64.whl (1.1 MB)


In [420]:
import json
from shapely.geometry import shape, GeometryCollection

freguesias_file = '../extras/geocoding/caop/freguesias.geojson'
with open(freguesias_file) as f:
  features = json.load(f)["features"]
print(len(features))

4050


In [442]:
from shapely.geometry import shape, Point
from shapely.geometry.multipolygon import MultiPolygon
from shapely.geometry.polygon import Polygon

def point_in_feature(longitude=None, latitude=None, features=None, property=None) -> dict:
    """ find features that contain a point.

    Args:
        longitude,latitude: coordinates of the point to locate
        features: list of features as GeoJSON dicts (Polygon or MultiPolygon)
        property: str,optional, name of feature property to return
                  if None return feature
        """
    for feature in features:
        geometry = feature["geometry"]
        gtype= geometry["type"]
        coordinates = geometry["coordinates"]
        # https://stackoverflow.com/questions/68820085/how-to-convert-geojson-to-shapely-polygon-solved
        boundary: dict = {'type': gtype,
                          'coordinates': coordinates}
        if gtype == 'MultiPolygon':
            fboundary: MultiPolygon = shape(boundary)
        elif gtype == 'Polygon':
            fboundary: Polygon = shape(boundary)
        else: 
            raise(ValueError("Only Polygon and MultiPolygon types accepted"))
        p: Point = Point(longitude,latitude) 
        if p.within(fboundary):
            if property is not None:
                if property not in feature['properties'].keys():
                    raise(ValueError("Property not present in feature"))
                return feature['properties'][property]
            else:
                return feature
    return None

In [588]:
places =ine_2011_pop[(ine_2011_pop.freg_cod.isnull()) & (ine_2011_pop.duplicate.isnull())][['freg_cod','origin','name','latitude','longitude']]
places

,freg_cod,origin,name,latitude,longitude


In [505]:
for n in places.index.values:
    lat = ine_2011_pop.iloc[n]['latitude']
    lng = ine_2011_pop.iloc[n]['longitude']
    print(ine_2011_pop.loc[n]['name'], end='')
    freg_cod = point_in_feature(lng,lat,features, property='DICOFRE')
    if freg_cod is not None:
        ine_2011_pop.loc[n,'freg_cod'] = freg_cod
        print(" Found: ",freg_cod)
    else:
        print(" Not found")


PÓVOA DE VARZIM Found:  131310
V. NOVA DE FAMALICÃO Found:  031248
GUIMARÃES Found:  030860
FAFE Found:  030709
BARCELOS Found:  030214
ESPOSENDE Found:  030605
CHAVES Found:  170350
VILA DO CONDE Found:  131628
SANTO TIRSO Found:  131422
FELGUEIRAS Found:  130320
AMARANTE Found:  130133
MAIA Found:  130607
MARCO DE CANAVESES Found:  130727
PAREDES Found:  131007
PAÇOS DE FERREIRA Found:  130912
PENAFIEL Found:  131124
VALONGO Found:  131505
PESO DA RÉGUA Found:  170807
VILA NOVA DE GAIA Found:  131716
GONDOMAR Found:  130409
MATOSINHOS Found:  130806
MIRANDELA Found:  040721
MIRANDA DO DOURO Found:  040608
MARINHA GRANDE Found:  101001
FIGUEIRA DA FOZ Found:  060511
ÁGUEDA Found:  010104
OLIVEIRA DE AZEMÉIS Found:  011309
OVAR Found:  011505
S. JOÃO DA MADEIRA Found:  011601
ESPINHO Found:  010702
SANTA MARIA DA FEIRA Found:  010906
ÍLHAVO Found:  011004
CANTANHEDE Found:  060204
POMBAL Found:  101509
LAMEGO Found:  180521
FUNDÃO Found:  050417
COVILHÃ Found:  050317
MANGUALDE Found: 

#### Completar o contexto administrativo a partir do código da freguesia

In [593]:
def lookup_in_df(column='code',value=None, table=None,subset=None):
    """Lookup a value in a dataframe, return a dict or single value
    Args:
        column: column where to lookupup
        value: value to lookup
        table: data frame to use as lookup table
        subset: string or list of columns to be returned
                If none all columns returned as dict.

    Example:
        lookup_in_df('Código','151206',cat2011_df,subset=['N3_DSG','COD_UPPER_LEVEL','COD_DSG'])
        > {'N3_DSG': 'São Simão', 'COD_UPPER_LEVEL': '1512','COD_DSG': 'São Simão, Setúbal, Setúbal'}
    """
    row = table[table[column]==value].iloc[0]
    if row is None:
        raise(ValueError("Code not found in categories"))
    else:
        if subset is None:
            return row.to_dict()
        elif type(subset) is list:
            return row[subset].to_dict()
        elif type(subset) is str:
            return row[subset]
        else:
            raise(ValueError("Subset must be list,str or None"))

In [482]:
ine_2011_pop.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36357 entries, 0 to 36356
Data columns (total 29 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  36357 non-null  object 
 1   name                36357 non-null  object 
 2   latitude            36357 non-null  float64
 3   longitude           36357 non-null  float64
 4   origin              36357 non-null  object 
 5   local_id            32507 non-null  object 
 6   inspire_id          29322 non-null  object 
 7   concelho_cod        25472 non-null  object 
 8   freg_cod            25545 non-null  object 
 9   sec_cod             25472 non-null  object 
 10  lug_cod             25472 non-null  object 
 11  distrito_code       25472 non-null  object 
 12  distrito_dsg        25472 non-null  object 
 13  concelho_dsg        25472 non-null  object 
 14  freg_dsg            25472 non-null  object 
 15  admin_dsg           25472 non-null  object 
 16  name

In [594]:
cols = ['distrito_code','distrito_dsg','concelho_cod','concelho_dsg','freg_dsg','admin_dsg']
subset=['N1_CODE','N1_DSG','N2_CODE','N2_DSG','N3_DSG','COD_DSG']

places = ine_2011_pop[ine_2011_pop.freg_cod.notnull() & (ine_2011_pop.name_dsg.isnull())]
for p,row in places.iterrows():
    ine_2011_pop.loc[p,cols] = lookup_in_df('Código',row.freg_cod,cat2011_df,subset=subset).values()
    name_dsg = row['name']+", "+ine_2011_pop.loc[p]['admin_dsg']
    ine_2011_pop.loc[p,'name_dsg'] = name_dsg

In [596]:
name_totals = ine_2011_pop.groupby(['name_dsg']).agg({'id':'count'}).sort_values('id', ascending=False)
repeated = name_totals[name_totals.id > 1]
repeated

,id
name_dsg,
"Olival, Olival, Vila Nova de Gaia, Porto",3
"Torreira, Torreira, Murtosa, Aveiro",3
"Mafra, Mafra, Mafra, Lisboa",3
"Venda do Alcaide, Palmela, Palmela, Setúbal",2
"Cesaredas, Reguengo Grande, Lourinhã, Lisboa",2
"Pedras Salgadas, Bornes de Aguiar, Vila Pouca de Aguiar, Vila Real",2
"Benavente, Benavente, Benavente, Santarém",2
"Benaciate, São Bartolomeu de Messines, Silves, Faro",2
"Ramalhosa, Sertã, Sertã, Castelo Branco",2


In [625]:
from itertools import combinations

for name in repeated.index.values:
    print(name, end=' ')
    name_group = ine_2011_pop[ine_2011_pop['name_dsg'] == name]
    ine_place = ine_2011_pop[(ine_2011_pop['name_dsg'] == name) 
                                & (ine_2011_pop['origin'] == 'INE')]
    if len(ine_place) == 0:
        print("no INE place involved: ",name,name_group.index.values)
        continue
    else:
        ine_place = ine_place.iloc[0].name # name here is the index

    print("ine place:",ine_place)
    pairs = list(combinations(name_group.index.values,2))
    # print(pairs)
    for l,r in pairs:
        if r == ine_place:
            l, r = r, l
        elif l != ine_place:
            # neither point in the INE place name
            continue
        left = ine_2011_pop.iloc[l]
        right = ine_2011_pop.iloc[r]
        dist_meters = geodesic((left.latitude,left.longitude), (right.latitude,right.longitude)).km*1000
        # print("distance:", (l,r),dist_meters)
        ine_2011_pop.loc[r,'duplicate'] = True
        ine_2011_pop.loc[r,'same_as'] = ine_2011_pop.iloc[l]['id']
        ine_2011_pop.loc[r,'same_as_score'] = 1
        ine_2011_pop.loc[r,'same_as_distance'] = dist_meters


Olival, Olival, Vila Nova de Gaia, Porto ine place: 9481
Torreira, Torreira, Murtosa, Aveiro ine place: 14802
Mafra, Mafra, Mafra, Lisboa ine place: 21012
Venda do Alcaide, Palmela, Palmela, Setúbal ine place: 21906
Cesaredas, Reguengo Grande, Lourinhã, Lisboa ine place: 20844
Pedras Salgadas, Bornes de Aguiar, Vila Pouca de Aguiar, Vila Real ine place: 14088
Benavente, Benavente, Benavente, Santarém ine place: 23018
Benaciate, São Bartolomeu de Messines, Silves, Faro ine place: 24890
Ramalhosa, Sertã, Sertã, Castelo Branco ine place: 19230
Branca, Branca, Coruche, Santarém ine place: 23133
Mértola, Mértola, Mértola, Beja ine place: 24124
Vinhais, Vinhais, Vinhais, Bragança ine place: 14260
Santa Cruz, Silveira, Torres Vedras, Lisboa ine place: 21346
Praia de Mira, Praia de Mira, Mira, Coimbra ine place: 15508
Belinho, Belinho, Esposende, Braga ine place: 7052
Albarraque, Rio de Mouro, Sintra, Lisboa ine place: 21600
Ponte Nova, São João, Ovar, Aveiro ine place: 14904
Avelãs de Caminho

## Cálculo da importância de cada topónimo

Usamos um índice misto, de importância administrativa e populacional.

Cada topónimo recebe um bonus de importância se o seu nome coincidir com o nome das circunscrições
administrativas em que se insere.

Quanto de maior nível for a circunscrição homónima maior o bonus (distrito=15, concelho=10, freguesia=5).

Adicionalmente cada lugar recebe um bonus proporcional à dimensão da sua população dentro 
de todos os lugares com o mesmo nome (1/rank). 

In [626]:
ine_2011_pop['pop_rank_in_name'] = ine_2011_pop[ine_2011_pop.residentes.notnull()].groupby('name')['residentes'].rank("dense", ascending=False)
ine_2011_pop.loc[ine_2011_pop.name == ine_2011_pop.distrito_dsg,'dist_bonus'] = 1
ine_2011_pop.loc[ine_2011_pop.name != ine_2011_pop.distrito_dsg,'dist_bonus'] = 0
ine_2011_pop.loc[ine_2011_pop.name == ine_2011_pop.concelho_dsg,'concelho_bonus'] = 1
ine_2011_pop.loc[ine_2011_pop.name != ine_2011_pop.concelho_dsg,'concelho_bonus'] = 0
ine_2011_pop.loc[ine_2011_pop.name == ine_2011_pop.freg_dsg,'freg_bonus'] = 1
ine_2011_pop.loc[ine_2011_pop.name != ine_2011_pop.freg_dsg,'freg_bonus'] = 0
if ine_2011_pop.pop_rank_in_name is not None:
    pop_rank = (1/ine_2011_pop.pop_rank_in_name)
else:
    pop_rank = 0
ine_2011_pop['importance'] = ine_2011_pop.dist_bonus * 15 + ine_2011_pop.concelho_bonus * 10 + ine_2011_pop.freg_bonus * 5 + pop_rank

In [627]:
ine_2011_pop[ine_2011_pop.name == 'Coimbra'].sort_values('importance', ascending=False)[['importance','origin','dgt_type','name','freg_cod', 'name_dsg','residentes','pop_rank_in_name']]

,importance,origin,dgt_type,name,freg_cod,name_dsg,residentes,pop_rank_in_name
15289,26.0,INE,NaN,Coimbra,060327,"Coimbra, Taveiro, Coimbra, Coimbra",105842.0,1.0
21127,0.5,INE,NaN,Coimbra,101402,"Coimbra, Atouguia da Baleia, Peniche, Leiria",490.0,2.0
30887,NaN,DGT,populatedPlace,Coimbra,NaN,NaN,NaN,NaN
31837,NaN,DGT,populatedPlace,Coimbra,060316,"Coimbra, Santa Clara, Coimbra, Coimbra",NaN,NaN


In [628]:
ine_2011_pop[ine_2011_pop.name == 'Granja'].sort_values('importance', ascending=False)[['importance','duplicate','name','freg_cod','name_dsg','residentes','pop_rank_in_name']]

,importance,duplicate,name,freg_cod,name_dsg,residentes,pop_rank_in_name
23899,5.500000,NaN,Granja,070801,"Granja, Granja, Mourão, Évora",587.0,2.0
13366,5.090909,NaN,Granja,170212,"Granja, Granja, Boticas, Vila Real",212.0,11.0
12797,5.076923,NaN,Granja,181204,"Granja, Granja, Penedono, Viseu",144.0,13.0
19846,5.058824,NaN,Granja,091309,"Granja, Granja, Trancoso, Guarda",127.0,17.0
21721,1.000000,NaN,Granja,111408,"Granja, Vialonga, Vila Franca de Xira, Lisboa",928.0,1.0
12429,0.333333,NaN,Granja,170101,"Granja, Alijó, Alijó, Vila Real",434.0,3.0
9347,0.250000,NaN,Granja,131604,"Granja, Azurara, Vila do Conde, Porto",354.0,4.0
15616,0.200000,NaN,Granja,061302,"Granja, Figueira de Lorvão, Penacova, Coimbra",319.0,5.0
13255,0.166667,NaN,Granja,171420,"Granja, Parada de Cunhos, Vila Real, Vila Real",306.0,6.0
15884,0.142857,NaN,Granja,100908,"Granja, Carvide, Leiria, Leiria",273.0,7.0


In [396]:
ine_2011_pop.sample(10)

,id,name,latitude,longitude,origin,local_id,inspire_id,concelho_cod,freg_cod,sec_cod,...,residentes,duplicate,same_as,same_as_score,same_as_distance,pop_rank_in_name,dist_bonus,concelho_bonus,freg_bonus,importance
24521,PT.GN.107498,Caliços,37.089714,-7.981599,INE,026118,BGRI2011_026118,0808,080801,080801003,...,233.0,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,1.000000
12089,PT.GN.95066,Moliceiro,41.003951,-8.485013,INE,011889,BGRI2011_011889,0109,010931,010931002,...,24.0,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,1.000000
23591,PT.GN.106568,Alter Pedroso,39.187979,-7.623203,INE,024993,BGRI2011_024993,1201,120101,120101003,...,48.0,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,1.000000
20051,PT.GN.103028,Fratel,39.628099,-7.746317,INE,020842,BGRI2011_020842,0511,051101,051101003,...,307.0,NaN,NaN,NaN,NaN,1.0,0.0,0.0,1.0,6.000000
30203,PT.GN.113182,Leião,38.728573,-9.295772,DGT,CDG200k_populatedPlace882,NaN,NaN,NaN,NaN,...,NaN,True,PT.GN.104522,0.957442,84.982817,NaN,0.0,0.0,0.0,NaN
29374,PT.GN.112353,Chaveca,37.070544,-7.903504,DGT,CDG200k_populatedPlace53,NaN,NaN,NaN,NaN,...,NaN,True,PT.GN.108365,0.990772,18.435511,NaN,0.0,0.0,0.0,NaN
4533,PT.GN.87510,Outeiro,42.002133,-8.494488,INE,000813,BGRI2011_000813,1604,160405,160405001,...,16.0,NaN,NaN,NaN,NaN,130.0,0.0,0.0,0.0,0.007692
20567,PT.GN.103544,Casal do Romanço,39.004409,-9.085017,INE,021410,BGRI2011_021410,1102,110202,110202001,...,180.0,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,1.000000
33168,PT.GN.116147,Vilar Torpim,40.826210,-6.951417,DGT,CDG200k_populatedPlace3994,NaN,NaN,NaN,NaN,...,NaN,True,PT.GN.102508,0.955314,89.372120,NaN,0.0,0.0,0.0,NaN
27253,PT.GN.110231,Belmonte Baixo,37.018589,-7.868579,INE,033188,BGRI2011_033188,0810,081004,081004006,...,159.0,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,1.000000


Reexportar com deteção de duplicados e importância

In [629]:

ine_topo_file = cache_file
ine_2011_pop.to_csv(ine_topo_file, index=False)
print("Exporting to", cache_file)

Exporting to ../extras/geocoding/INE/ine_toponimia_clipped.csv


In [361]:
import folium
from folium.plugins import MarkerCluster
from geopy.distance import geodesic

uc_location = [40.207422, -8.4260033]
centro_pt = [39.694502, -8.130573]

cell = folium.Figure(width=950, height=900)
map = folium.Map(location=centro_pt,
                zoom_start=7, 
                control_scale=True,
                scrollWheelZoom=True).add_to(cell)
marker_cluster = MarkerCluster()
# for icons check https://github.com/lennardv2/Leaflet.awesome-markers
marker_types = { 'INE': 'bookmark',
                 'DGT':'home',
                 'CIGEOE': 'flag'}
marker_colors = ['black', 'red', 'blue', 'green', 'purple',
                 'orange', 'darkred', 'lightred', 'beige',
                 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple',
                 'pink', 'lightblue', 'lightgreen', 'gray', 'lightgray']

head = 5
show =  candidates[-5:]
i = 1
for (l,r), same_name,same_location,total in show.itertuples():
    print(l,r)
    left = ine_2011_pop.iloc[l]
    right = ine_2011_pop.iloc[r]
    left_marker = marker_types[left.origin]
    right_marker = marker_types[right.origin]
    color = marker_colors[i % len(marker_colors)]
    dist_meters = geodesic((left.latitude,left.longitude), (right.latitude,right.longitude)).km*1000

    folium.Marker(location=[left.latitude,left.longitude],
                   tooltip=f"{left['name']} {left.admin_dsg}",
                   icon=folium.Icon(color=color,
                   prefix='glyphicon', icon=left_marker)).add_to(map)
    folium.Marker(location=[right.latitude,right.longitude],
                   tooltip=f"{right['name']} {right.admin_dsg}",
                   icon=folium.Icon(color=color,
                   prefix='glyphicon', icon=right_marker)).add_to(map)
    folium.PolyLine([(left.latitude,left.longitude),(right.latitude,right.longitude)],
                    tooltip=f"{dist_meters} ({same_location})",
                    color='red',
                    opacity=1
                    ).add_to(map)
    # print(f"{left}={right} {total} {left.origin} {right.origin} {color}")
    i = i + 1
# marker_cluster.add_to(map)
map

13667 35859
16420 31114
12455 35964
15529 31984
11004 34518


Definimos um pop-up em html

In [112]:
def pop_info(cluster,name,origin,id, context):
    styles="""
    <!DOCTYPE html>
    <html>
    <head>
    <style>
    table, th, td {
    border: 1px solid;
    }
    table {
     width: 100%;
     border-collapse: collapse;
    }
    </style>
    </head>
    <body>"""
    
    html=f"<table><tr><td width=50%><b>{name}</b></td><td>C{cluster}</td><td>{origin}</td><td>id:{id}</td></tr>"
    html=f'{styles}{html}<tr><td colspan=4>{context}</td></tr></table>'
    return html

In [131]:
import folium
from folium.plugins import MarkerCluster

uc_location = [40.207422, -8.4260033]
centro_pt = [39.694502, -8.130573]

cell = folium.Figure(width=950, height=900)
map = folium.Map(location=centro_pt,
                zoom_start=7, 
                control_scale=True,
                scrollWheelZoom=True).add_to(cell)

ncolors = len(clusters)
marker_colors = ['black', 'red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple',  'pink', 'lightblue', 'lightgreen', 'gray', 'lightgray']
color_list = marker_colors

while len(color_list) < ncolors:
    color_list.extend(marker_colors[1:])

place_colors = dict()
i = 1
for cluster in clusters:
    for place in cluster:
        place_colors[place] = i
    i = i+1
print(place_colors)

# for icons check https://github.com/lennardv2/Leaflet.awesome-markers
marker_types = { 'INE': 'bookmark',
                 'DGT':'home',
                 'CIGEOE': 'flag'}

marker_cluster = MarkerCluster()
for i, name, origin, lat, lng, id, name_dsg,sameas, duplicate in ine_2011_pop.loc[ocorrencias.index.unique()][['name','origin','latitude','longitude','id','name_dsg','same_as','duplicate']].itertuples():
    color = place_colors.get(i,0)
    if duplicate:
        name_dsg = "duplicado de: "+sameas
    # print(i, color,name, origin, lat, lng, id)
    marker = marker_types.get(origin,'square-question')
    iframe = folium.IFrame(pop_info(color,name,origin,id,name_dsg),width=400,height=70)
    popup = folium.Popup(iframe,max_width=500)
    folium.Marker(location=[lat,lng],popup=popup,
                    icon=folium.Icon(color=marker_colors[color],
                    prefix='glyphicon', icon=marker)).add_to(marker_cluster)

marker_cluster.add_to(map)
map

{14265: 1, 1425: 1, 36337: 1, 14745: 2, 1054: 2, 3282: 3, 14949: 3, 4042: 4, 24766: 4, 16274: 5, 32548: 5, 33107: 6, 26478: 6, 33256: 7, 26093: 7, 12788: 8, 34286: 8, 11210: 9, 34420: 9, 13041: 10, 34570: 10, 12922: 11, 34629: 11, 11611: 12, 34780: 12, 34897: 13, 29847: 13, 10689: 14, 35062: 14, 35120: 15, 12263: 15, 12203: 15, 12375: 15, 11668: 16, 35124: 16, 35138: 17, 12174: 17, 27404: 18, 35351: 18, 35458: 19, 10237: 19, 10324: 20, 35470: 20, 35481: 21, 9385: 21, 35608: 22, 9393: 22, 35656: 23, 14031: 23, 8653: 24, 35726: 24, 7905: 25, 35743: 25, 35842: 26, 7524: 26, 7296: 27, 35917: 27, 9866: 28, 35942: 28, 8646: 29, 35974: 29, 10130: 30, 36006: 30, 10151: 30, 36027: 31, 9643: 31, 9854: 31, 36029: 32, 7206: 32, 8080: 33, 36327: 33, 7113: 34, 36413: 34, 7810: 35, 36429: 35, 7450: 36, 36443: 36, 36472: 37, 9179: 37, 9211: 37, 7744: 38, 36477: 38, 36542: 39, 7935: 39, 7641: 40, 36546: 40, 30104: 41, 6906: 41, 36607: 41, 36635: 42, 7309: 42, 6002: 43, 6355: 43, 36989: 43, 5417: 44, 37